In [ ]:
!pip install razdel
!pip3 install -r "/content/drive/MyDrive/Colab Notebooks/dialogue_simplification/requirements.txt"
!pip install rnnmorph

!pip install git+https://github.com/huggingface/transformers

!git clone https://github.com/feralvam/easse.git
%cd /content/easse/
!pip install .


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks'
!git clone https://github.com/in-oleynikov/dialogue-rusimplification.git 


/content/drive/MyDrive/Colab Notebooks
Cloning into 'dialogue-rusimplification'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


## Loading data

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook
from razdel import tokenize
from functools import lru_cache
from rnnmorph.predictor import RNNMorphPredictor
from sklearn.model_selection import train_test_split
from easse.sari import corpus_sari
import json
PATH = '/content/drive/MyDrive/Colab Notebooks/dialogue_simplification'

In [ ]:
# Read the data
df = pd.read_csv(os.path.join(PATH,'dev_sents.csv'))
pd.set_option('display.max_colwidth', 150)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3406 non-null   int64 
 1   INPUT:source   3406 non-null   object
 2   OUTPUT:output  3406 non-null   object
dtypes: int64(1), object(2)
memory usage: 80.0+ KB


In [ ]:
df.head()

,Unnamed: 0,INPUT:source,OUTPUT:output
0,3,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
1,4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси"
2,5,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,В выпущенной в 1960 году модель имела изменения в хвостовом оперении и руле направления
3,6,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: в хвосте и руле направления с обратной стреловидностью и крепления для поплавкового шасси.\r\n
4,7,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"Модель 172А с другим хвостовым оперением, креплениями для поплавкового шасси рулём направления с обратной стреловидностью появилась в 1960 году."


## Text preprocess

Группируем данные по INPUT:





In [ ]:
df_grouped = df.groupby('INPUT:source' ).agg({'OUTPUT:output': '(SEP)'.join})
input_corpus = df_grouped.index.values
output_corpus = [output_text.split('(SEP)') for output_text in df_grouped['OUTPUT:output'].values]


In [ ]:
df_grouped.head(5)

,OUTPUT:output
INPUT:source,
1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси(SEP)В выпущенной в 1960 году модель имела изме..."
"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
"26 августа 2014 года Болдок принял участие в памятном матче Кубка Футбольной лиги против «Манчестер Юнайтед», в котором «донс» сенсационно обыграл команду Луи ван Гала со счётом 4:0.",26 августа 2014 года Болдок выиграл футбольный матч Кубка лиги у команды Луи ван Гала со счётом 4:0(SEP)26 августа 2014 года Болдок принял участие...
"Armory — клиент с различными функциями для повышения безопасности, является надстройкой, работающей поверх bitcoind, однако использует собственный формат хранения секретного ключа.","Armory - программа с различными функциями для повышения безопасности. Является дополнением, которое работает вместе с bitcoind, но использует свой..."
"Concert for Diana (рус. Концерт для Дианы) — концерт памяти, организованный в честь Дианы, принцессы Уэльской.","Concert for Diana - это концерт, посвящённый памяти принцессы Дианы Уэльской.(SEP)В честь Дианы, принцессы Уэльской ,организован концерт памяти Ди..."


In [ ]:
#@lru_cache(maxsize=32)
def text_preprocess(corpus):
    predictor = RNNMorphPredictor(language="ru")
    new_corpus = []
    for sentence in tqdm_notebook(corpus, desc= 'preprocessing...'):
        sentence = ' '.join([predictor.predict([token.text])[0].normal_form 
                             if token.text.isnumeric()==False else token.text 
                             for token in list(tokenize(sentence))])
        new_corpus.append(sentence)
    return new_corpus


In [ ]:
def dataframe_lemmatized(input_corpus, output_corpus):
    df_lemmatized = pd.DataFrame()
    input_text_lemmatized = text_preprocess(input_corpus)
    # returns list of lemmatized group of text which corresponds to one input text
    output_text_lemmatized = [text_preprocess(group) for group in output_corpus]

    df_lemmatized['input'] = input_text_lemmatized 
    df_lemmatized = pd.concat([df_lemmatized , pd.DataFrame(data = output_text_lemmatized, columns = ['output_1' , 
                                                                                                      'output_2' , 
                                                                                                      'output_3' , 
                                                                                                      'output_4' , 
                                                                                                      'output_5'])], axis=1)
    return df_lemmatized

In [ ]:
df_lemmatized = dataframe_lemmatized(input_corpus, output_corpus)

In [ ]:
df_lemmatized.to_csv(os.path.join(PATH, 'lemmatized_text.csv'))

In [ ]:
try:
    df_lemmatized
except NameError :
    df_lemmatized = pd.read_csv(os.path.join(PATH,'lemmatized_text.csv'))
df_lemmatized = df_lemmatized.fillna(0)

In [ ]:
df_lemmatized.head()

,Unnamed: 0,input,output_1,output_2,output_3,output_4,output_5
0,0,1960 год быть выпущенный модель 172 a . изменение : хвостовой оперение и руль направление с обратный стреловидность и крепление для поплавковый ша...,"в 1960 год выйти модель 172 а . отличие в хвостовой оперение , в руль направление и в крепление шасси",в выпущенный в 1960 год модель иметь изменение в хвостовой оперение и руль направление,изменение : в хвост и руль направление с обратный стреловидность и крепление для поплавковый шасси .,"модель 172 а с другой хвостовой оперение , крепление для поплавковый шасси руль направление с обратный стреловидность появиться в 1960 год .",0
1,1,"14 декабрь 1944 год рабочий посёлок ичка быть переименованный в рабочий посёлок советский , посол что поселковый совет стать называться советский .",14 декабрь 1944 год рабочий посёлок ичка переименованный в советский .,0,0,0,0
2,2,"26 август 2014 год болдок принять участие в памятный матч кубок футбольный лига против « манчестер юнайтед » , в который « донс » сенсационно обыг...",26 август 2014 год болдок выиграть футбольный матч кубок лига у команда луи ван гала со счёт 4 : 0,"26 август 2014 год болдок принять участие в футбольный матч против « манчестер юнайтед » , где победить со счёт 4 : 0 .",0,0,0
3,3,"armory — клиент с различный функция для повышение безопасность , являться надстройка , работающий поверх bitcoind , однако использовать собственны...","armory - программа с различный функция для повышение безопасность . являться дополнение , который работать вместе с bitcoind , но использовать сво...",0,0,0,0
4,4,"concert for diana ( рус . концерт для диана ) — концерт память , организованный в честь диана , принцесса уэльский .","concert for diana - это концерт , посвящённый память принцесса диана уэльский .","в честь диана , принцесса уэльский , организованный концерт память диана .","концерт для дианы- концентр память диана , принцесса уэльский .",0,0


In [ ]:
df_lemmatized.shape

(1000, 7)

## Matching

Загружаем лемматизированный текст:

In [ ]:
input = df_grouped.index.values
output = output_corpus
input_lemma = df_lemmatized['input'].values
output_lemma = df_lemmatized[['output_1' , 'output_2' , 'output_3' , 'output_4' , 'output_5'] ].values


Разбиваем датасет на тренировочную, валидационную и тестовую выборки

In [ ]:
np.random.seed=42

ids = np.arange(1000)
train_id, valid_id = train_test_split(ids, train_size = 0.7 )
valid_id, test_id = train_test_split(valid_id, train_size = 0.5)


train_input, train_input_lemma, train_output_lemma = input[train_id], input_lemma[train_id] , output_lemma[train_id]
valid_input, valid_input_lemma, valid_output_lemma = input[valid_id], input_lemma[valid_id] , output_lemma[valid_id]
test_input, test_input_lemma, test_output_lemma = input[test_id], input_lemma[test_id] , output_lemma[test_id]

print('Train data size:', train_input.shape , train_input_lemma.shape , train_output_lemma.shape )
print('Valid data size:', valid_input.shape , valid_input_lemma.shape , valid_output_lemma.shape )
print('Test data size:', test_input.shape , test_input_lemma.shape , test_output_lemma.shape )


Train data size: (700,) (700,) (700, 5)
Valid data size: (150,) (150,) (150, 5)
Test data size: (150,) (150,) (150, 5)


Записываем токены и тэги в JSON-файл в BIO формате:

In [ ]:
#{"tokens": ["APPLICATION", "and", "Affidavit", "for", "Search", "Warrant", "as", "to", "The", "Matter", "of", "the", "Search", "of", "9", "Granite", "Street", ",", "#", "5", "(", "Attachments", ":", "#", "1", "Affidavit", "of", "James", "Keczkemethy)(Belpedio", ",", "Lisa", ")", "(", "Entered", ":", "12/15/2020", ")"], "tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "I-MISC", "L-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}

def matching(input, input_lemma, output_lemma, train_or_test):    
    result_json = []
    with open(os.path.join(PATH,f'{train_or_test}.json') , 'w') as file:        
        for i, sentence in enumerate(input_lemma):
            token_tag_dict = {}
            tags = [] 

            input_lemma_sentence = input_lemma[i].split(' ')  
            input_sentence = [token.text for token in list(tokenize(input[i]))]       
            token_tag_dict['tokens'] = input_sentence
          
            for token in sentence.split(' '):
                
                # Объединяем выходны варианты в один список через запятую
                output_lemma_sentence = ' '.join(output_lemma[i][output_lemma[i]!=0])
                output_lemma_sentence = [token.text for token in list(tokenize(output_lemma_sentence))]
           
                if token in output_lemma_sentence:
                    tags.append('I-YES')                              
                else:  
                    tags.append('O')                
            token_tag_dict['tags'] = tags         
      
            json.dump(token_tag_dict , file, ensure_ascii = False)
            file.write('\n')
         
    #return result_json



In [ ]:
# train 70% of data
matching(train_input, train_input_lemma, train_output_lemma, 'train')

# valid 15% of data
matching(valid_input, valid_input_lemma, valid_output_lemma, 'valid')

# test 15% of data
matching(test_input, test_input_lemma, test_output_lemma, 'test')


## Training

In [ ]:
!wget "https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py"

--2021-03-19 12:06:11--  https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘run_ner.py.1’

run_ner.py.1            [ <=>                ] 260.90K  1.63MB/s    in 0.2s    

2021-03-19 12:06:12 (1.63 MB/s) - ‘run_ner.py.1’ saved [267163]



In [ ]:
!export MAX_LENGTH=128
!export BERT_MODEL=bert-base-multilingual-cased
!export OUTPUT_DIR=germeval-model
!export BATCH_SIZE=32
!export NUM_EPOCHS=3
!export SAVE_STEPS=750
!export SEED=1


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/dialogue_simplification


/content/drive/MyDrive/Colab Notebooks/dialogue_simplification


In [ ]:
!python ./run_ner.py \
  --model_name_or_path bert-base-multilingual-cased \
  --pad_to_max_length=True \
  --train_file train.json \
  --validation_file valid.json \
  --test_file test.json \
  --output_dir OUTPUT_DIR \
  --do_train \
  --do_eval \
  --do_predict

2021-03-19 15:21:30.539198: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/19/2021 15:21:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/19/2021 15:21:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=OUTPUT_DIR, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Mar19_15-21-32_52987434817d, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, 

## Predictions

In [ ]:
def tag_to_text(path_to_file): 
    simplified = []
    with open(path_to_file, 'rb') as file:
        for i,row in enumerate(file):
            tags = np.array(str(row)[2:-3].split(' '))
            I_YES_id, = np.where(tags=='I-YES')
            
            test_tokens_list = np.array([token.text for token in list(tokenize(test_input[i]))])
            
            test_simple = test_tokens_list[I_YES_id]
            test_simple = ' '.join(test_simple.tolist())
            simplified.append(test_simple)
    return simplified

In [ ]:

simplified = tag_to_text('./OUTPUT_DIR/test_predictions.txt')

0
["b'I-YES" 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES'
 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'O' 'I-YES'
 'I-YES' 'I-YES']
1
["b'O" 'O' 'O' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'O' 'O' 'I-YES' 'I-YES' 'O'
 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES']
2
["b'I-YES" 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES'
 'O' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES']
3
["b'I-YES" 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES'
 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES']
4
["b'O" 'I-YES' 'O' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES'
 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'O' 'O' 'O'
 'I-YES' 'I-YES' 'I-YES' 'O' 'O' 'I-YES' 'O' 'O' 'O' 'I-YES' 'I-YES' 'O'
 'O' 'O' 'O']
5
["b'I-YES" 'O' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'O' 'O' 'I-YES' 'I-YES'
 'I-YES' 'I-YES' 'O' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES']
6
["b'I-YES" 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES' 'I-YES

In [ ]:
len([token.text for token in list(tokenize(test_output_lemma[33]))])

TypeError: ignored

In [ ]:
test_output_lemma[33]

array(['в свой воспоминание жюля верна описывать детский восторг от набережный луара и суд , проплывавший мимо дача он семья в деревня шантен .',
       0, 0, 0, 0], dtype=object)

## Evaluation

Data for evaluation

In [ ]:
input_sents = test_input.tolist()
output_sents = np.array(output_corpus)[test_id].tolist()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=input_sents,  
            sys_sents=simplified, 
            refs_sents= output_sents)

10.048914127283073

In [ ]:
input_sents[33]

'В своих воспоминаниях «фр. Souvenirs d’enfance et de jeunesse» Жюль Верн описывал детский восторг от набережной Луары, проплывавших торговых судов мимо деревушки Шантене, где его отец приобрёл дачу.'

In [ ]:
simplified[33]

'В воспоминаниях enfance et jeunesse Жюль Верн описывал детский восторг от набережной Луары , торговых судов деревушки Шантене , его отец приобрёл дачу .'

In [ ]:
output_sents[33]

['В своих воспоминаниях Жюль Верн описывал детский восторг от набережной Луары и судов, проплывавших мимо дачи его семьи в деревне Шантене. ']

In [11]:
%cd '/content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification'
!git init
!git config — global user.email "inoleynikov@yandex.ru"
!git config — global user.name “in-oleynikov”
!git add dialogue_simplification.ipynb
!git commit -m "something with result"
!git remote add origin https://in-oleynikov:  

/content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone      Clone a repository into a new directory
   init       Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add        Add file contents to the index
   mv         Move or rename a file, a directory, or a symlink
   reset      Reset current HEAD to the specified state
   rm         Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect     Use binary s